In [1]:
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup

In [81]:
address = pd.read_excel("C:/Users/ICT01_14/Desktop/cctv주소.xlsx")

In [82]:
address.head()

,주소,카메라합계,스피드돔 카메라,검지 카메라,번호인식카메라
0,제주시 (동문로9길 3) 건입동 1120,5,1,4,0
1,제주시 화북동 1933-9,2,0,2,0
2,제주시 한림읍 동명리,4,1,3,0
3,제주시 한림읍 대림리 1698-4,10,0,10,0
4,제주시 조천읍 조천리 2690,2,1,1,0


In [83]:
address.shape

(1662, 5)

In [5]:
# 괄호제거하기
import re
#s = " 제주 (강정동) 조천읍"
#re.sub(pattern="\([^)]*\)", repl="", string=s).strip() # 양 공백제거

#re.sub(pattern="\([^)]*\)", repl="", string=s) # 공백제거 안한거

In [6]:
address['addr'] = address['주소'].apply(lambda x: re.sub(pattern="\([^)]*\)", repl="", string=x).strip() )

In [7]:
address.drop(['스피드돔 카메라', '검지 카메라', '번호인식카메라','주소'], axis=1, inplace=True)

In [8]:
address.head()

,카메라합계,addr
0,5,제주시 건입동 1120
1,2,제주시 화북동 1933-9
2,4,제주시 한림읍 동명리
3,10,제주시 한림읍 대림리 1698-4
4,2,제주시 조천읍 조천리 2690


# Geocoder 로 좌표값으로 바꾸기

In [ ]:
# 예시 #Geocoder ApI 2.0
ApiKey="CBB6D3FC-B604-3380-A1FB-84E02062F9CA"
address="제주시 일도2동 1146-1"

apiUrl= "http://api.vworld.kr/req/address"

#service=address&request=getCoord&key=CBB6D3FC-B604-3380-A1FB-84E02062F9CA"
    
values = {
    'address':address,
    'type':'PARCEL',
    'service' : 'address',
    'request' : 'getCoord',
    'key' : 'CBB6D3FC-B604-3380-A1FB-84E02062F9CA'
}

x = requests.get(url=apiUrl, params=values).json()  #>> point 에 위도경도나온다. 
x
x['response']['result']['point']['x']  # 위도에 접근

In [ ]:
ApiKey="C622680F-FCA8-348E-8ED5-54B10E8EEA3C"

add= address['addr']
apiUrl= "http://api.vworld.kr/req/address"
    
values = {
    'type':'PARCEL',
    'service' : 'address',
    'request' : 'GetCoord',
    'key' : 'C622680F-FCA8-348E-8ED5-54B10E8EEA3C'
}

result = [] # 주소를 찾은 것들
result_x= [] # 위도 주소
result_y = [] # 경도 주소

except_list = [] # 주소를 찾지못하는 것들

In [ ]:
for i in add:  # 각각 하나씩 가져온다.
    values['address'] =i  # address 키값을 각각 추가시켜준다.
    try:
        x = requests.get(url=apiUrl, params=values).json()
        print(i, x['response']['result']['point'])
        result.append(i)
        result_x.append(x['response']['result']['point']['x'])
        result_y.append(x['response']['result']['point']['y'])


        
    except:
        except_list.append(i)
        continue

        
#result_df : 주소 찾은 것들 데이터프레임
result_df = pd.DataFrame({'cctv_addr':result, \
                            "latitude":result_x, \
                            'longitude':result_y})

In [ ]:
result_df

In [ ]:
except_list

In [ ]:
ApiKey="C622680F-FCA8-348E-8ED5-54B10E8EEA3C"

add= except_list
apiUrl= "http://api.vworld.kr/req/address"
    
values = {
    'type':'ROAD', # road 로 하면 도로명 주소 가능 
    'service' : 'address',
    'request' : 'GetCoord',
    'key' : 'C622680F-FCA8-348E-8ED5-54B10E8EEA3C'
}

result2 = [] # 주소를 찾은 것들(도로명)
result2_x= [] # 위도 주소
result2_y = [] # 경도 주소

except2_list = [] # 주소를 찾지못하는 것들

In [ ]:
# 도로명 주소 였던 것들 다시뽑아내기

In [ ]:
for i in add:  # 각각 하나씩 가져온다.
    values['address'] =i  # address 키값을 각각 추가시켜준다.
    try:
        x = requests.get(url=apiUrl, params=values).json()
        print(i, x['response']['result']['point'])
        result2.append(i)
        result2_x.append(x['response']['result']['point']['x'])
        result2_y.append(x['response']['result']['point']['y'])


        
    except:
        except2_list.append(i)
        continue

        
#result_df : 주소 찾은 것들 데이터프레임
result2_df = pd.DataFrame({'cctv_addr':result2, \
                            "latitude":result2_x, \
                            'longitude':result2_y})

In [ ]:
# 동현오빠가 알려준 주소랑, 위도 경도 뽑는 것
def find_address(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?"
    apikey = "f7b706e030ca85d4e6874a24943d2110"
    query = address
    r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )
#     js = simplejson.JSONEncoder().encode(r.json())
    r_json = r.json()
    ad = r_json['documents'][0]
    address = ad['address_name']
    x = ad['x']
    y = ad['y']
    data = [address, x, y]
    return data

# 카카오 Api 활용해서 위도 경도 구하기

In [ ]:
!pip install simplejson

In [9]:
import simplejson,requests

def find_places_keyword(searching):
    url = "https://dapi.kakao.com/v2/local/search/address.json?" # address라쓰면 : 주소 , keyword라쓰면 : keyword 를 주소로 인식
    apikey = "285ed6abcb6d080e51cd530a0249669d"
    query = searching
    r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )
    js = simplejson.JSONEncoder().encode(r.json())
    r_json = r.json()
    
    return r_json
    
#     print(r_json)
#     rj = r_json['documents'][0]
#     name = rj['place_name']
#     x = rj['x']
#     y = rj['y']
#     data = [name, x, y, searching]
#     return data

In [10]:
address.head()

,카메라합계,addr
0,5,제주시 건입동 1120
1,2,제주시 화북동 1933-9
2,4,제주시 한림읍 동명리
3,10,제주시 한림읍 대림리 1698-4
4,2,제주시 조천읍 조천리 2690


In [11]:
result = []  # api 로 주소를 찾은 것
result_x = [] # 위도 주소
result_y = [] # 경도 주소

except_list = [] # 주소를 찾지못하는 것들

In [12]:
for i in address['addr']:
    try:
        print("주소는 : ", i,"위도는 : ", find_places_keyword(i)['documents'][0]['address']['x'], \
             "경도는 : ", find_places_keyword(i)['documents'][0]['address']['y'])
        
        result.append(i)
        result_x.append(find_places_keyword(i)['documents'][0]['address']['x'])
        result_y.append(find_places_keyword(i)['documents'][0]['address']['y'])
        
    except:
        except_list.append(i)
        continue

주소는 :  제주시  건입동 1120 위도는 :  126.529969111718 경도는 :  33.5133161037981
주소는 :  제주시 화북동 1933-9 위도는 :  126.564971826229 경도는 :  33.5231149791422
주소는 :  제주시 한림읍 동명리 위도는 :  126.271436820871 경도는 :  33.3974353440562
주소는 :  제주시 한림읍 대림리 1698-4 위도는 :  126.273996126978 경도는 :  33.4204944166691
주소는 :  제주시 조천읍 조천리 2690 위도는 :  126.635975929695 경도는 :  33.5401954944093
주소는 :  제주시 조천읍 조천리 2373 위도는 :  126.636403307219 경도는 :  33.5365827197338
주소는 :  제주시 조천읍 신촌리 2462 위도는 :  126.616803791388 경도는 :  33.5351811542386
주소는 :  제주시 조천읍 신촌리 2156-1 위도는 :  126.62056494667 경도는 :  33.5364682164846
주소는 :  제주시 이도1동 1421-3 위도는 :  126.5255545112 경도는 :  33.5103158218536
주소는 :  제주시 용담1동 298-1 위도는 :  126.515823038485 경도는 :  33.5115301189742
주소는 :  제주시 용담1동 위도는 :  126.513266470185 경도는 :  33.5090977985233
주소는 :  제주시 오라2동 652-2 위도는 :  126.508386819953 경도는 :  33.4666279217418
주소는 :  제주시 애월읍 하가리 977 위도는 :  126.352443629482 경도는 :  33.457659174948
주소는 :  제주시 애월읍 하가리 872 위도는 :  126.351524735454 경도는 :  33.4562427559506
주소는 :  제주시 애월읍 고성

주소는 :  제주시 조천읍 북촌1길 43앞 위도는 :  126.684491479384 경도는 :  33.5480331690617
주소는 :  제주시 조천읍 북촌12길 25앞 위도는 :  126.698755805317 경도는 :  33.5470325848947
주소는 :  제주시 조천읍 대흘리2351-4 위도는 :  126.652591877059 경도는 :  33.4986379485946
주소는 :  제주시 조천읍 남조로3길 44앞 위도는 :  126.647397996073 경도는 :  33.520583007472
주소는 :  제주시 제주대학로4길 16 위도는 :  126.55428826609 경도는 :  33.4566194264667
주소는 :  제주시 일도2동404-9 위도는 :  126.54378853831 경도는 :  33.5064123477466
주소는 :  제주시 일도2동 995-16 위도는 :  126.532456946088 경도는 :  33.512088355442
주소는 :  제주시 일도2동 984-12 위도는 :  126.536828650897 경도는 :  33.5123939177652
주소는 :  제주시 일도2동 84-1 위도는 :  126.546076905115 경도는 :  33.4998607637885
주소는 :  제주시 일도2동 84 위도는 :  126.545625327024 경도는 :  33.4996184733684
주소는 :  제주시 일도2동 81 위도는 :  126.544853746206 경도는 :  33.5007051266913
주소는 :  제주시 일도2동 46-2 위도는 :  126.552254209589 경도는 :  33.5033957577269
주소는 :  제주시 일도2동 46-2 위도는 :  126.552254209589 경도는 :  33.5033957577269
주소는 :  제주시 일도2동 46-2 위도는 :  126.552254209589 경도는 :  33.5033957577269
주소는 :  제주시 일도2동 46-2 위

주소는 :  제주시 연수로 4길 1 위도는 :  126.543268856982 경도는 :  33.5028824115902
주소는 :  제주시 연삼로 489 위도는 :  126.540363055981 경도는 :  33.5012716092828
주소는 :  제주시 연동334 위도는 :  126.491466637415 경도는 :  33.4824349349619
주소는 :  제주시 연동261-42 위도는 :  126.488168928831 경도는 :  33.4885594907354
주소는 :  제주시 연동261 A 위도는 :  126.488959758356 경도는 :  33.4879243371972
주소는 :  제주시 연동2412-1 제주서중 지하보도 위도는 :  126.480487552555 경도는 :  33.4936738993403
주소는 :  제주시 연동 292-94 위도는 :  126.493401623116 경도는 :  33.4881403841226
주소는 :  제주시 연동 1546 위도는 :  126.492853991271 경도는 :  33.4792878382926
주소는 :  제주시 연동 1488 위도는 :  126.491305626241 경도는 :  33.4808792064748
주소는 :  제주시 연동 1374 위도는 :  126.491146879978 경도는 :  33.4814948838751
주소는 :  제주시 애월읍 하귀2리 1950-3 위도는 :  126.399931652306 경도는 :  33.4770802456733
주소는 :  제주시 애월읍 하귀2리 1899-7 위도는 :  126.402771339765 경도는 :  33.4773057444224
주소는 :  제주시 애월읍 하귀1리 46 위도는 :  126.419908695755 경도는 :  33.4863847226052
주소는 :  제주시 애월읍 하귀1리 215-1 위도는 :  126.412833531333 경도는 :  33.4811044302619
주소는 :  제주시 애월읍 하가리 163

주소는 :  제주시 동광로 21길 5 위도는 :  126.542225833419 경도는 :  33.5097678984698
주소는 :  제주시 동광로 20길 9 위도는 :  126.544087691198 경도는 :  33.507179261361
주소는 :  제주시 동광로 20길 28 위도는 :  126.54281149402 경도는 :  33.5086102535043
주소는 :  제주시 독짓골 8길 19 위도는 :  126.532390698285 경도는 :  33.4923051875662
주소는 :  제주시 도평동 1405 위도는 :  126.452051177905 경도는 :  33.4745493162095
주소는 :  제주시 도평동 1403 위도는 :  126.450262935282 경도는 :  33.4789437787182
주소는 :  제주시 도평동 1403 위도는 :  126.450262935282 경도는 :  33.4789437787182
주소는 :  제주시 도련2동 576-1 위도는 :  126.590296627422 경도는 :  33.514656852415
주소는 :  제주시 도련2동 448 위도는 :  126.59231077528 경도는 :  33.5141518432304
주소는 :  제주시 도련1동 2302-4 위도는 :  126.583023485367 경도는 :  33.5073459965685
주소는 :  제주시 도련1동 1654 위도는 :  126.589128370792 경도는 :  33.5091009699866
주소는 :  제주시 도련1동 1027-1 위도는 :  126.592926129594 경도는 :  33.5009079130309
주소는 :  제주시 도두1동 2439-1 위도는 :  126.465473616225 경도는 :  33.5024426433722
주소는 :  제주시 도남로4길 3 위도는 :  126.525476946462 경도는 :  33.4973723648391
주소는 :  제주시 도남동 924-9번지 위도는 :  126.52

주소는 :  제주시  오라2동 2178-1 연삼로길 옆 위도는 :  126.511121298614 경도는 :  33.4924942103807
주소는 :  제주시  건입동 1237-1 위도는 :  126.530907493019 경도는 :  33.5147030611252
주소는 :  제주시  이도2동 1060-3 위도는 :  126.535950308126 경도는 :  33.4970018963596
주소는 :  제주시  일도2동 141-26 위도는 :  126.54261428982 경도는 :  33.5126261159947
주소는 :  제주시  오라2동 3828 위도는 :  126.513995633542 경도는 :  33.468550242385
주소는 :  위미리 4614 위도는 :  126.648534817614 경도는 :  33.2703940713666
주소는 :  위미리 4200-2 위도는 :  126.649282706961 경도는 :  33.2687851487619
주소는 :  오라2동 3827-6 위도는 :  126.505465724831 경도는 :  33.4865373956931
주소는 :  영평동 2506-5 위도는 :  126.560004465166 경도는 :  33.4828573314346
주소는 :  연동 283-29 위도는 :  126.493154472596 경도는 :  33.4876589150237
주소는 :  서귀포시 회수동 441-5 위도는 :  126.443432057419 경도는 :  33.2631481868865
주소는 :  서귀포시 호근동 2142 위도는 :  126.534468379269 경도는 :  33.2569004409225
주소는 :  서귀포시 호근동 1756-3 위도는 :  126.52830476083 경도는 :  33.2557472942497
주소는 :  서귀포시 호근동 1170-2 위도는 :  126.529595935324 경도는 :  33.2613606677872
주소는 :  서귀포시 호근동 1110-1 위도는 :  

...

주소는 :  제주시 조천읍 조천리 3140 위도는 :  126.633805206966 경도는 :  33.5354653849872
주소는 :  제주시 조천읍 함덕리 287-2 위도는 :  126.672266844911 경도는 :  33.5385725629281
주소는 :  제주시 조천읍 대흘리 1651-4 위도는 :  126.658666803567 경도는 :  33.4988285702921
주소는 :  제주시 조천읍 신촌리 1859-1 위도는 :  126.618591738054 경도는 :  33.5346301212754
주소는 :  제주시 조천읍 함덕리 72-34 위도는 :  126.67277078273 경도는 :  33.5374676408549
주소는 :  제주시 한경면 고산리 2684-1 위도는 :  126.177196684867 경도는 :  33.3042172478998
주소는 :  제주시 한경면 저지리 1519-3 위도는 :  126.252462779163 경도는 :  33.3262036764675
주소는 :  제주시 한경면 고산리 2205-2 위도는 :  126.179441079936 경도는 :  33.3035044111813
주소는 :  제주시 한경면 저지리 1520-5 위도는 :  126.252785215055 경도는 :  33.3266034019228
주소는 :  제주시 한림읍 귀덕리 1157 위도는 :  126.293799509819 경도는 :  33.4422566609656
주소는 :  제주시 한림읍 금악리 1898-3 위도는 :  126.295677604995 경도는 :  33.3572718344928
주소는 :  제주시 한림읍 귀덕리 1020-3 위도는 :  126.293356808584 경도는 :  33.4435902888898
주소는 :  제주시 한림읍 협재리 2472 위도는 :  126.237378979683 경도는 :  33.3903653820003
주소는 :  제주시 건입동 671 위도는 :  126.540021607309 경도는 

주소는 :  서귀포시 상예동 1170-1 위도는 :  126.39309041155 경도는 :  33.2471928285448
주소는 :  제주시 오라2동 872-1 위도는 :  126.511618089248 경도는 :  33.4761070877175
주소는 :  제주시 외도1동 634-10 위도는 :  126.429090565236 경도는 :  33.4856251280454
주소는 :  제주시 외도1동 435-5 위도는 :  126.433042553334 경도는 :  33.4917567631692
주소는 :  제주시 도평동 587-10 위도는 :  126.450969099407 경도는 :  33.490666771336
주소는 :  제주시 도평동 1269 위도는 :  126.443575679925 경도는 :  33.4790929030661
주소는 :  제주시 용담2동 608-3 위도는 :  126.505583792697 경도는 :  33.5117084931529
주소는 :  제주시 우도면 연평리 1442 위도는 :  126.95524500255 경도는 :  33.5054028719085
주소는 :  제주시 우도면 연평리 1442 위도는 :  126.95524500255 경도는 :  33.5054028719085
주소는 :  제주시 이도1동 1692-22 위도는 :  126.52775599572 경도는 :  33.5056277546553
주소는 :  제주시 이도2동 404 위도는 :  126.527577275521 경도는 :  33.4796234965667
주소는 :  제주시 이도2동 375-1 위도는 :  126.523573500664 경도는 :  33.4793544619974
주소는 :  제주시 이도2동 320-6 위도는 :  126.530259892602 경도는 :  33.4811637901719
주소는 :  제주시 이도2동 2024-1 위도는 :  126.535511005352 경도는 :  33.4865483423783
주소는 :  제주시 이도2동 2012

주소는 :  제주시 도련2동 578-5 위도는 :  126.589909381247 경도는 :  33.5149493101221
주소는 :  제주시 도련1동 1969-4 위도는 :  126.579481354898 경도는 :  33.5151320843933
주소는 :  제주시 삼양1동 1593-1 위도는 :  126.593781041924 경도는 :  33.5290959682707
주소는 :  제주시 삼양1동 1647-12 위도는 :  126.590513413346 경도는 :  33.5260532462241
주소는 :  제주시 삼양2동 2100-2 위도는 :  126.581977244578 경도는 :  33.5242251542806
주소는 :  제주시 삼양2동 2104-4 위도는 :  126.584962879536 경도는 :  33.5243640320752
주소는 :  제주시 삼양2동 2113-1 위도는 :  126.585668916693 경도는 :  33.5245241744092
주소는 :  제주시 삼양2동 2130-1 위도는 :  126.583180865647 경도는 :  33.5221303052132
주소는 :  제주시 삼양2동 2154-1 위도는 :  126.580091398326 경도는 :  33.5200833546615
주소는 :  제주시 삼양2동 2179-10 위도는 :  126.583921949437 경도는 :  33.5202271018702
주소는 :  제주시 삼양2동 2315 위도는 :  126.58329586351 경도는 :  33.5187333129506
주소는 :  제주시 삼양2동 2321 위도는 :  126.579332428617 경도는 :  33.5192157921011
주소는 :  제주시 삼양2동 2329 위도는 :  126.585625763353 경도는 :  33.5177042975229
주소는 :  서귀포시 서홍동 454-3 위도는 :  126.555645335242 경도는 :  33.2501493359269
주소는 :  서귀포시 서

주소는 :  대정읍 구억리 1143앞 글로벌에듀로 위도는 :  126.286589287559 경도는 :  33.2862004540142
주소는 :  대정읍 구억리 1133앞 국제학교 정문쪽 위도는 :  126.283494962935 경도는 :  33.2875355166852
주소는 :  대정읍 구억리 1059공원앞 글로벌에듀로 145번길 위도는 :  126.288890720491 경도는 :  33.295708014757
주소는 :  대정읍 구억리 1090공원앞 위도는 :  126.291867917048 경도는 :  33.2958870331876
주소는 :  대정읍 구억리 1124공원 위도는 :  126.287021375693 경도는 :  33.2891304523145
주소는 :  대정읍 구억리 1129공원 위도는 :  126.284871662346 경도는 :  33.2899411789676
주소는 :  대정읍 구억리 1185 위도는 :  126.287948754898 경도는 :  33.2883301826246
주소는 :  서귀포시 강정동 4445-18 위도는 :  126.479452768441 경도는 :  33.2357770242395
주소는 :  서귀포시 도순동 269-4 위도는 :  126.478316683362 경도는 :  33.2504792397243
주소는 :  서귀포시 동홍동 1514-6 위도는 :  126.563890752927 경도는 :  33.2569796065457
주소는 :  서귀포시 동홍동 1383-2 위도는 :  126.56216156517 경도는 :  33.2602926067522
주소는 :  서귀포시 동홍동 1473-7 위도는 :  126.562528313831 경도는 :  33.2561730020879
주소는 :  제주시 삼도2동 958-2 위도는 :  126.521204285026 경도는 :  33.5131720028901
주소는 :  제주시 삼도2동 16-4 위도는 :  126.523302966357 경도는 :  33.51557

주소는 :  제주시 노형동 3799 위도는 :  126.476980661893 경도는 :  33.4810331703894
주소는 :  제주시 도두1동 2605-7 위도는 :  126.466937089344 경도는 :  33.5046754213282
주소는 :  제주시 애월읍 하귀1리 199 위도는 :  126.413863118297 경도는 :  33.4848829916761
주소는 :  제주시 용담2동 924-3 위도는 :  126.503345083563 경도는 :  33.5122954887737
주소는 :  제주시 이도2동 1042-2 위도는 :  126.518836011835 경도는 :  33.489842887838
주소는 :  제주시 이도2동 1928-15 위도는 :  126.551423227459 경도는 :  33.4910952812335
주소는 :  제주시 아라1동 6055 위도는 :  126.541130383792 경도는 :  33.4731320709986
주소는 :  제주시 아라1동 6125 위도는 :  126.550687447777 경도는 :  33.4730029014662
주소는 :  제주시 아라1동 6071 위도는 :  126.54291822786 경도는 :  33.4718935710357
주소는 :  제주시 아라1동 6058-6 위도는 :  126.541087932593 경도는 :  33.4711849150059
주소는 :  제주시 아라1동 6077 위도는 :  126.544799637638 경도는 :  33.4695806841664
주소는 :  제주시 아라1동 6108-3 위도는 :  126.54647634214 경도는 :  33.4760031367674
주소는 :  제주시 아라1동 6111-1 위도는 :  126.549121493137 경도는 :  33.4765328281517
주소는 :  제주시 아라1동 6131-3 위도는 :  126.54855740294 경도는 :  33.4727602058321
주소는 :  제주시 아라1동 6141

In [116]:
# 주소있는 dataframe
result_df = pd.DataFrame({'cctv_addr':result, \
                            "latitude":result_x, \
                            'longitude':result_y})

NameError: name 'result' is not defined

In [28]:
result_df

,cctv_addr,latitude,longitude
0,제주시 건입동 1120,126.529969111718,33.5133161037981
1,제주시 화북동 1933-9,126.564971826229,33.5231149791422
2,제주시 한림읍 동명리,126.271436820871,33.3974353440562
3,제주시 한림읍 대림리 1698-4,126.273996126978,33.4204944166691
4,제주시 조천읍 조천리 2690,126.635975929695,33.5401954944093
...,...,...,...
1452,서귀포시 중문동 2439-14,126.430739343786,33.2497349080267
1453,서귀포시 신효동 123-9,126.616162123009,33.265083674975
1454,서귀포시 신효동 123-17,126.6151648971,33.2646801784271
1455,서귀포시 안덕면 창천리 92-6,126.368504443459,33.2653212412794


In [53]:
len(except_list)

205

In [54]:
result2 = []  # api 로 주소를 찾은 것
result2_x = [] # 위도 주소
result2_y = [] # 경도 주소

except2_list = [] # 주소를 찾지못하는 것들

In [55]:
# 주소 못찾은것 -> keyword 로 바꿔서 다시 찾기
def find_places_keyword2(searching):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?" # address라쓰면 : 주소 , keyword라쓰면 : keyword 를 주소로 인식
    apikey = "285ed6abcb6d080e51cd530a0249669d"
    query = searching
    r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )
    js = simplejson.JSONEncoder().encode(r.json())
    r_json = r.json()
    
    return r_json

In [56]:
for i in except_list:
    try:
        print("주소는 : ", i,"위도는 : ", find_places_keyword2(i)['documents'][0]['x'], \
             "경도는 : ", find_places_keyword2(i)['documents'][0]['y'])
        
        result2.append(i)
        result2_x.append(find_places_keyword2(i)['documents'][0]['x'])
        result2_y.append(find_places_keyword2(i)['documents'][0]['y'])
        
    except:
        except2_list.append(i)
        continue

주소는 :  서귀포시 안덕면 사계리 3126 위도는 :  126.288384000828 경도는 :  33.2395439886881
주소는 :  제주시 조천읍 조천리1355 위도는 :  126.641445504835 경도는 :  33.5341305745302
주소는 :  제주시 연동 146 위도는 :  126.493838170305 경도는 :  33.4224198927301
주소는 :  제주시 애월읍 구엄리 86 위도는 :  126.38260688615952 경도는 :  33.47661207377783
주소는 :  제주시 신설동길 위도는 :  126.548565685701 경도는 :  33.4964667010706
주소는 :  제주시 관덕로 17길 위도는 :  126.52832511352166 경도는 :  33.51492893115256
주소는 :  제주시  시청 대학로 위도는 :  126.531304128312 경도는 :  33.498809864106
주소는 :  서귀포시 토평동 2269 위도는 :  126.585013336636 경도는 :  33.2668687421016
주소는 :  서귀포시 대정읍 동일리 2659 위도는 :  126.239092885166 경도는 :  33.2322088331743
주소는 :  서귀포시 남원읍 신례리 822 위도는 :  126.632082180874 경도는 :  33.2845626432054
주소는 :  서귀포시 남원읍 신례리 822 위도는 :  126.632082180874 경도는 :  33.2845626432054
주소는 :  서귀포시 신효동 123 위도는 :  126.616148682669 경도는 :  33.2645740319978
주소는 :  서귀포시 하원동 323 위도는 :  126.459437948984 경도는 :  33.2531575853017
주소는 :  제주시 구좌읍 세화리 1391 위도는 :  126.858942391813 경도는 :  33.5227980107783
주소는 :  제주시 애월읍 하가리 1580

주소는 :  제주 함덕초등학교 위도는 :  126.672407482774 경도는 :  33.5388216846248
주소는 :  제주 해안초등학교 위도는 :  126.45410318437 경도는 :  33.4568114092028
주소는 :  제주 화북초등학교 위도는 :  126.561736242351 경도는 :  33.5186589096898
주소는 :  제주 효돈초등학교 위도는 :  126.616148682669 경도는 :  33.2645740319978
주소는 :  제주 흥산초등학교 위도는 :  126.765284243239 경도는 :  33.3001303185819
주소는 :  제주 함덕초등학교 위도는 :  126.672407482774 경도는 :  33.5388216846248
주소는 :  제주 동복분교 위도는 :  126.713524942265 경도는 :  33.5523501582046
주소는 :  제주 도련초등학교 위도는 :  126.583976408421 경도는 :  33.5137299578683
주소는 :  제주 더럭초등학교 위도는 :  126.345460621561 경도는 :  33.4536239642503
주소는 :  제주 조천교래분교 위도는 :  126.678061957447 경도는 :  33.4364651752945
주소는 :  제주 함덕초등학교 위도는 :  126.672407482774 경도는 :  33.5388216846248


In [57]:
len(except2_list)

69

In [59]:
result2_df = pd.DataFrame({'cctv_addr':result2, \
                            "latitude":result2_x, \
                            'longitude':result2_y})

In [ ]:
#  남은 except2_list 주소 구하기

In [70]:
ApiKey="C622680F-FCA8-348E-8ED5-54B10E8EEA3C"

add = except2_list
apiUrl= "http://api.vworld.kr/req/address"
    
values = {
    'type':'PARCEL',
    'service' : 'address',
    'request' : 'GetCoord',
    'key' : 'C622680F-FCA8-348E-8ED5-54B10E8EEA3C'
}

result3 = [] # 주소를 찾은 것들
result3_x= [] # 위도 주소
result3_y = [] # 경도 주소

except3_list = [] # 주소를 찾지못하는 것들

In [71]:
for i in add:  # 각각 하나씩 가져온다.
    values['address'] =i  # address 키값을 각각 추가시켜준다.
    try:
        x = requests.get(url=apiUrl, params=values).json()
        print(i, x['response']['result']['point'])
        result3.append(i)
        result3_x.append(x['response']['result']['point']['x'])
        result3_y.append(x['response']['result']['point']['y'])


        
    except:
        except3_list.append(i)
        continue


서귀포시 남원읍 신흥리 21-3 {'x': '126.731288955433', 'y': '33.3477789499253'}
서귀포시 법환동 4 {'x': '126.513764259756', 'y': '33.2176871793866'}


In [72]:
result3_df = pd.DataFrame({'cctv_addr':result3, \
                            "latitude":result3_x, \
                            'longitude':result3_y})
result3_df

,cctv_addr,latitude,longitude
0,서귀포시 남원읍 신흥리 21-3,126.731288955433,33.3477789499253
1,서귀포시 법환동 4,126.513764259756,33.2176871793866


In [74]:
len(except3_list)

67

In [75]:
ApiKey="C622680F-FCA8-348E-8ED5-54B10E8EEA3C"

add = except3_list
apiUrl= "http://api.vworld.kr/req/address"
    
values = {
    'type':'ROAD',
    'service' : 'address',
    'request' : 'GetCoord',
    'key' : 'C622680F-FCA8-348E-8ED5-54B10E8EEA3C'
}

result4 = [] # 주소를 찾은 것들
result4_x= [] # 위도 주소
result4_y = [] # 경도 주소

except4_list = [] # 주소를 찾지못하는 것들

In [76]:
for i in add:  # 각각 하나씩 가져온다.
    values['address'] =i  # address 키값을 각각 추가시켜준다.
    try:
        x = requests.get(url=apiUrl, params=values).json()
        print(i, x['response']['result']['point'])
        result4.append(i)
        result4_x.append(x['response']['result']['point']['x'])
        result4_y.append(x['response']['result']['point']['y'])


        
    except:
        except4_list.append(i)
        continue


제주시 신선동길16 {'x': '126.531484702', 'y': '33.509998337'}
서귀포시 신서로52번길 대신중학교 정문 {'x': '126.510373286', 'y': '33.252789191'}


In [77]:
result4_df = pd.DataFrame({'cctv_addr':result4, \
                            "latitude":result4_x, \
                            'longitude':result4_y})
result4_df

,cctv_addr,latitude,longitude
0,제주시 신선동길16,126.531484702,33.509998337
1,서귀포시 신서로52번길 대신중학교 정문,126.510373286,33.252789191


In [84]:
# 합치기
data = pd.concat([result_df, result2_df, result3_df, result4_df],axis=0)

In [85]:
data

,cctv_addr,latitude,longitude
0,제주시 건입동 1120,126.529969111718,33.5133161037981
1,제주시 화북동 1933-9,126.564971826229,33.5231149791422
2,제주시 한림읍 동명리,126.271436820871,33.3974353440562
3,제주시 한림읍 대림리 1698-4,126.273996126978,33.4204944166691
4,제주시 조천읍 조천리 2690,126.635975929695,33.5401954944093
...,...,...,...
135,제주 함덕초등학교,126.672407482774,33.5388216846248
0,서귀포시 남원읍 신흥리 21-3,126.731288955433,33.3477789499253
1,서귀포시 법환동 4,126.513764259756,33.2176871793866
0,제주시 신선동길16,126.531484702,33.509998337


In [91]:
data.to_excel("C:/Users/ICT01_14/Desktop/cctv_addr.xlsx")

In [92]:
except4_list

['제주시 일도2동 1146',
 '제주시 삼도1동 970-2',
 '서귀포시 서귀동 227-1',
 '서귀포시 상효동 1156-6',
 '제주시 한림읍 청수리 1730-7',
 '제주시 표선면 가시리 682-1',
 '제주시 표선면 가시리 2703-1',
 '제주시 조천읍 와흘시 산12-7',
 '제주시 일도2동 1107-12',
 '제주시 용담2동 2818-3',
 '제주시 영평동 2407-3',
 '제주시 연북신설길18',
 '제주시 애월읍 상귀리 1260',
 '제주시 애월읍 고성리 1178-16',
 '제주시 아라1동 2196',
 '제주시 삼도2동 1228-28',
 '제주시 건입동 956-1',
 '제주시 건입동 956-1',
 '제주시 516도로 3120',
 '제주시  오라2동 1454-6',
 '서귀포시 세화1리  삼거리',
 '서귀포시 성산읍 삼달리 1065-5',
 '서귀포시 서홍동 705-12',
 '서귀포시 남원읍 태흥리480-4',
 '서귀포시 남원읍 남원리 205-18',
 '서귀포시  신효동 548-11',
 '서귀포시 동홍동 2145-2',
 '서귀포시 성산읍 고성리 1123-3',
 '제주시 노형동 1390',
 '제주시 노형동 1390',
 '제주시 외도 2동 253',
 '제주시 외도 2동 253',
 '서귀포시 서홍동 372-1',
 '서귀포시 서호동 1420-1\n서귀포시 서호동 1421-5',
 '제주시 애월읍 광령1리 1333-12',
 '제주시 아라1동 1646-2',
 '제주시 아라1동 1670-1',
 '서귀포시 서귀포시 서귀동 586',
 '제주시 아라1동 2196',
 '서귀포시 서호동 689',
 '서귀포시 대정읍 보성리 106',
 '서귀포시 대정읍 구억리 627-1',
 '서귀포시 대정읍 구억리 638',
 '서귀포시 대정읍 구억리 산11',
 '서귀포시 대정읍 보성리 산4',
 '제주시 아라1동 868-1',
 '제주시 아라1동 1600-4',
 '서귀포시 애월읍 수산리 2093-8',
 '서귀포시 

In [ ]:
# api 로 주소 바꾼 것 + 도청에서 바꿔준 것 합치기

In [1]:
import pandas as pd

In [66]:
cctv_addr = pd.read_excel("C:/Users/ICT01_14/Desktop/cctv_addr.xlsx", usecols='B:D')

In [67]:
cctv_addr.head()

,cctv_addr,latitude,longitude
0,제주시 건입동 1120,126.529969,33.513316
1,제주시 화북동 1933-9,126.564972,33.523115
2,제주시 한림읍 동명리,126.271437,33.397435
3,제주시 한림읍 대림리 1698-4,126.273996,33.420494
4,제주시 조천읍 조천리 2690,126.635976,33.540195


In [112]:
cctv_addr2 = pd.read_excel("C:/Users/ICT01_14/Desktop/cctv_addr2.xlsx", header=0)
cctv_addr2.columns = ['cctv_addr', 'location']

In [113]:
cctv_addr2.head()

,cctv_addr,location
0,제주시 일도2동 1146,"33.511802, 126.526085"
1,제주시 삼도1동 970-2,"33.512015, 126.521756"
2,서귀포시 서귀동 227-1,"33.249184, 126.564014"
3,서귀포시 상효동 1156-6,"33.320578, 126.593605"
4,제주시 한림읍 청수리 1730-7,"33.323106, 126.248884"


In [ ]:
df = pd.read_csv("복사본 CCTV 주소 이상한것_수정 (1).csv", encoding = 'utf-8')
df.columns = ['주소', '좌표']
df['좌표'] = df.좌표.str.split(',')
df['x'] = 100
df['y'] = 200
df.to_csv("location.csv", encoding = 'utf-8')